In [65]:
import sys, os, io, json
from neo4j import GraphDatabase
from py2neo import Graph
from pathlib import Path
from pandas import DataFrame
import networkx as nx

graph = Graph("bolt://localhost:7687")
driver = GraphDatabase.driver('bolt://localhost:7687', auth=None)



## Get the neighbourhood of Fipronil as GraphML

In [27]:
df = DataFrame(graph.run("""
WITH $query AS query
CALL apoc.export.graphml.query(query, null, {stream:true, useTypes:true, storeNodeIds:true, readLabels:true})
YIELD file, source, format, nodes, relationships, properties, time, rows, batchSize, batches, done, data
RETURN data;
""", {'query': """
MATCH (other)-[r]-(fipronil:GraphNode)-[:id]->(id:Id { id: \"chebi:5063\" })
RETURN *
"""}).data())

with open('fipronil.graphml', 'w') as f:
    f.write(df['data'][0])



## Loading into networkx and writing Cytoscape JSON

In [79]:

## Helper function to run a Cypher query over a GrEBI graph and return a NetworkX graph from the results
##
def query_to_nx(query):
    results = driver.session().run(query)
    G = nx.MultiDiGraph()
    for node in list(results.graph()._nodes.values()):
        if not 'Id' in node._labels:
            G.add_nodes_from([(node._properties['grebi:nodeId'], dict({"grebi:type": list(node._labels)}, **node._properties))])
    for rel in list(results.graph()._relationships.values()):
        if not 'Id' in rel.start_node._labels and not 'Id' in rel.end_node._labels:
            G.add_edges_from([(rel.start_node._properties['grebi:nodeId'], rel.end_node._properties['grebi:nodeId'], rel._properties['edge_id'], dict({"grebi:type": rel.type}, **rel._properties))])
    return G

G = query_to_nx("""
MATCH (other)-[r]-(fipronil:GraphNode)-[:id]->(id:Id { id: \"chebi:5063\" })
RETURN *
""")

with open('fipronil.cyjs.json', 'w') as f:
    cydata = nx.cytoscape_data(G)
    for node in cydata['elements']['nodes']:
        node['data']['ids'] = node['data']['id']
        node['data']['id'] = node['data']['grebi:nodeId']
        del node['data']['grebi:nodeId']
    json.dump(cydata, f, indent=2)



